In [ ]:
import ipywidgets as widgets
from IPython.display import display
import oracledb
from queue import Queue
from typing import List, Dict
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# Database Configuration Class
@dataclass
class DBConfig:
    username: str
    password: str
    host: str
    port: int
    service_name: str
    
    def get_dsn(self) -> str:
        return f"{self.host}:{self.port}/{self.service_name}"

# Oracle Year Partitioned Query Class
class OracleYearPartitionedQuery:
    def __init__(self, db_config: DBConfig, start_year: int = 2006, end_year: int = 2023, thick_mode: bool = False):
        self.db_config = db_config
        self.start_year = start_year
        self.end_year = end_year
        self.result_queue = Queue()
        
        if thick_mode:
            oracledb.init_oracle_client()
    
    def get_connection(self):
        return oracledb.connect(
            user=self.db_config.username,
            password=self.db_config.password,
            dsn=self.db_config.get_dsn()
        )
    
    def process_year(self, base_query: str, year: int) -> List[Dict]:
        wrapped_query = f"""
            WITH filtered_data AS (
                {base_query}
            )
            SELECT *
            FROM filtered_data
            WHERE YR = {year}
        """
        results = []
        try:
            with self.get_connection() as connection:
                with connection.cursor() as cursor:
                    cursor.execute(wrapped_query)
                    columns = [col[0] for col in cursor.description]
                    
                    batch_size = 10000
                    total_rows = 0
                    while True:
                        rows = cursor.fetchmany(batch_size)
                        if not rows:
                            break
                        batch_results = [dict(zip(columns, row)) for row in rows]
                        results.extend(batch_results)
                        total_rows += len(batch_results)
                        print(f"Year {year}: Processed {total_rows} rows so far...")
                            
            print(f"Completed Year {year}: Total {len(results)} records")
            
        except Exception as e:
            print(f"Error processing year {year}: {str(e)}")
            raise
        
        self.result_queue.put((year, results))
        return results

    def execute_parallel(self, query: str, max_concurrent_threads: int = 4) -> List[Dict]:
        years = list(range(self.start_year, self.end_year + 1))
        total_years = len(years)
        
        print(f"Starting parallel execution for years {self.start_year} to {self.end_year}")
        print(f"Using maximum {max_concurrent_threads} concurrent threads")
        
        start_time = time.time()
        
        with ThreadPoolExecutor(max_workers=max_concurrent_threads) as executor:
            futures = [
                executor.submit(self.process_year, query, year)
                for year in years
            ]
            
            for future in futures:
                try:
                    future.result()
                except Exception as e:
                    print(f"Thread execution failed: {str(e)}")
        
        all_results = []
        temp_results = []
        
        while not self.result_queue.empty():
            temp_results.append(self.result_queue.get())
        
        temp_results.sort(key=lambda x: x[0])
        
        print("\nProcessing Summary:")
        print("-" * 40)
        for year, results in temp_results:
            print(f"Year {year}: {len(results):,} records")
            all_results.extend(results)
        
        end_time = time.time()
        total_time = end_time - start_time
        
        print("\nPerformance Metrics:")
        print("-" * 40)
        print(f"Total Processing Time: {total_time:.2f} seconds")
        print(f"Average Time per Year: {total_time/total_years:.2f} seconds")
        print(f"Total Records: {len(all_results):,}")
        
        return all_results

# Widgets for user input
username = widgets.Text(description="Username", value="your_username")
password = widgets.Password(description="Password", value="your_password")
host = widgets.Text(description="Host", value="your_host")
port = widgets.IntText(description="Port", value=1521)
service_name = widgets.Text(description="Service Name", value="your_service")

start_year = widgets.IntSlider(value=2006, min=2000, max=2023, description="Start Year")
end_year = widgets.IntSlider(value=2023, min=2000, max=2023, description="End Year")
max_threads = widgets.IntSlider(value=4, min=1, max=10, description="Threads")

query_input = widgets.Textarea(
    value="""
        SELECT t.*, EXTRACT(YEAR FROM your_date_column) as YR
        FROM your_table t
        WHERE your_conditions = 1
    """,
    description="Query",
    layout=widgets.Layout(width="80%", height="150px")
)

run_button = widgets.Button(description="Run Query")
output = widgets.Output()

# Function to run the query
def run_query(button):
    with output:
        output.clear_output()
        
        # Create the DB config
        db_config = DBConfig(
            username=username.value,
            password=password.value,
            host=host.value,
            port=port.value,
            service_name=service_name.value
        )
        
        # Initialize query processor
        threaded_query = OracleYearPartitionedQuery(
            db_config=db_config,
            start_year=start_year.value,
            end_year=end_year.value
        )
        
        # Execute query
        try:
            results = threaded_query.execute_parallel(query_input.value, max_concurrent_threads=max_threads.value)
            
            # Convert results to DataFrame and display
            if results:
                df = pd.DataFrame(results)
                display(df)
            else:
                print("No results returned.")
        
        except Exception as e:
            print(f"An error occurred: {str(e)}")

# Attach the button to the run query function
run_button.on_click(run_query)

# Display the widgets
display(username, password, host, port, service_name, start_year, end_year, max_threads, query_input, run_button, output)
